In [1]:
import requests

Set up your API token from your profile on observation portal

In [2]:
API_TOKEN = 'XXX'
PROPOSAL_ID = 'XXX'

In [13]:
# Determine when the observations are to take place
# The period is NOT the period of the variable. 
# The period is the cadence at which you want to observe, in HOURS
# The Jitter is the window within which your observation may take place. 

period = 20./60
jitter = 20./60
cadence = {
    'start': '2019-09-29 22:00:00',
    'end': '2019-09-30 10:00:00',
    'period': period,
    'jitter': jitter
}

In [14]:
target = {
    'name': 'Target_B',
    'type': 'ICRS',
    'ra': 42.44194,
    'dec': -38.42737,
    'epoch': 2000
}

In [15]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

constraints = {
    'max_airmass': 1.6,
    'min_lunar_distance': 20
}

In [16]:
# This does not need to be changed.
# Exposure time is in SECONDS

configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'v'
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'b'
                }
            }
        ]
    }
]

In [17]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
windows = []

# The telescope class that should be used for this observation
location = {
    'telescope_class': '0m4'
}

In [18]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
requestgroup = {
    'name': 'TestData_Target_B',  # The title
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': cadence,
        'configurations': configurations,
        'windows': windows,
        'location': location,
    }]
}

In [19]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=requestgroup  # Make sure you use json!
)

In [20]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise ex

In [21]:
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

Cadence generated 35 requests
Request 1 window start: 2019-09-29T22:00:00Z window end: 2019-09-29T22:10:00Z
Request 2 window start: 2019-09-29T22:10:00Z window end: 2019-09-29T22:30:00Z
Request 3 window start: 2019-09-29T22:30:00Z window end: 2019-09-29T22:50:00Z
Request 4 window start: 2019-09-29T22:50:00Z window end: 2019-09-29T23:10:00Z
Request 5 window start: 2019-09-29T23:10:00Z window end: 2019-09-29T23:30:00Z
Request 6 window start: 2019-09-29T23:30:00Z window end: 2019-09-29T23:50:00Z
Request 7 window start: 2019-09-29T23:50:00Z window end: 2019-09-30T00:10:00Z
Request 8 window start: 2019-09-30T00:10:00Z window end: 2019-09-30T00:30:00Z
Request 9 window start: 2019-09-30T00:30:00Z window end: 2019-09-30T00:50:00Z
Request 10 window start: 2019-09-30T00:50:00Z window end: 2019-09-30T01:10:00Z
Request 11 window start: 2019-09-30T01:10:00Z window end: 2019-09-30T01:30:00Z
Request 12 window start: 2019-09-30T01:30:00Z window end: 2019-09-30T01:50:00Z
Request 13 window start: 2019-0

In [28]:
# Submit the fully formed RequestGroup
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

Total number of observing hours needed for this request:  3.2472222222222222


# Do not run the following cell until you have confirmed with an instructor your observation.

In [12]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure this API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()

# Print out the url on the portal where we can view the submitted request
print('View this observing request: https://observe.lco.global/requestgroups/{}/'.format(requestgroup_dict['id']))

View this observing request: https://observe.lco.global/requestgroups/858381/
